It is useful to have summary files that allow you to quickly answer questions such as the following:
•	How have our sales-by-day changed over the last few months?
•	What is our most popular item in each department?
•	Which owners spend the most per month in each department?
The classic way to structure data to answer these questions is in a relational database. In this task, you will build the summary text files that hold this data and populate a relational database with the data.
Input
You will process your owner records in GBQ to build the summary tables.
Output
For this task, you will build a single SQLite database via Python (in a .db file) containing three tables:
1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items . 
2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 
3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.
You will submit your Python code that builds the database. 
You are welcome to generate these tables via queries in Google Big Query, export the text files, and store them locally on your machine. Then you will need to write a Python script that creates the database, creates the tables, and fills those tables. Obviously, it’d be great to do the whole thing in Python. 


In [1]:
import os
import pandas as pd
import numpy as np
import pandas_gbq
import sqlite3
import db_dtypes

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# These first two values will be different on your machine. 
service_path = ""
service_file = 'umt-msba-gg-key.json' # change this to your authentication information  

gbq_proj_id = 'umt-msba'  

# And this should stay the same. 
private_key = service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(private_key)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [3]:
con = sqlite3.connect('reporting.db')

In [4]:
query = """
    SELECT
  extract(date from datetime) AS Date,
  extract(hour from datetime) AS Hour,
  Round(SUM(total)) AS Spend,
  SUM(CASE When trans_status in ("V", "R") then -1 else 1 end) AS Items,

  COUNT(Distinct(
    CONCAT(
      extract(date from datetime), 
      register_no,
      emp_no,
      trans_no
    )
  )) AS trans


FROM
  `umt-msba.transactions.transArchive_*`
WHERE
  department NOT IN (0,
    15)
  AND (trans_status IS NULL
    OR trans_status IN ("V",
      "R",
      " "))
Group BY Date, Hour
ORDER BY Date, Hour

"""

The where clause is used to remove transactions that are in departments 0 because it does not actually exist and 15 which misc P/I not an actual department. For rows where the trans status is null v, r, or " " because these are coupons and returns and we don't want them being taken from our total.

In [5]:
holder = pandas_gbq.read_gbq(query,project_id=gbq_proj_id,credentials=credentials)

Downloading: 100%|██████████|


In [6]:
holder.head()

,Date,Hour,Spend,Items,trans
0,2010-01-01,9,1006.0,245,36
1,2010-01-01,10,3129.0,913,82
2,2010-01-01,11,4002.0,1108,118
3,2010-01-01,12,3887.0,1143,124
4,2010-01-01,13,4655.0,1365,154


In [7]:
holder.to_sql("date_hour",con,if_exists="replace",index=False)

39330

In [8]:
#Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.
query_2 = '''
Select
  card_no,
  extract(year from datetime) AS year,
  extract(month from datetime) AS month,
  Round(SUM(total)) AS Sales,
  SUM(CASE When trans_status in ("V", "R") then -1 else 1 end) AS Items,

  COUNT(Distinct(
    CONCAT(
      extract(date from datetime), 
      register_no,
      emp_no,
      trans_no
    )
  )) AS trans

FROM
  `umt-msba.transactions.transArchive_*`
WHERE
  department NOT IN (0,
    15)
  AND (trans_status IS NULL
    OR trans_status IN ("V",
      "R",
      " "))
Group by card_no, year, month
Order by card_no, year, month



'''

In [9]:
holder2 = pandas_gbq.read_gbq(query_2,project_id=gbq_proj_id,credentials=credentials)

holder2.head()

Downloading: 100%|██████████|


,card_no,year,month,Sales,Items,trans
0,3.0,2016,10,0.0,226859,26939
1,3.0,2012,2,0.0,263393,25127
2,3.0,2014,10,0.0,240412,24871
3,3.0,2015,12,0.0,212741,23997
4,3.0,2011,11,0.0,261003,25567


In [10]:
holder2.to_sql("owner_sales",con,if_exists="replace",index=False)

810178

In [11]:
#Sales by product description by year by month: 
# A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

query_3 = """
SELECT
  t.upc
  ,t.description
  ,t.department
  ,d.dept_name
  ,extract(year from datetime) AS year
  ,extract(month from datetime) As month
  ,Round(sum(total),2) As sales
  ,SUM(CASE When trans_status in ("V", "R") then -1 else 1 end) AS Items
  ,COUNT(Distinct(
    CONCAT(
      extract(date from datetime), 
      register_no,
      emp_no,
      trans_no
    )
  )) AS trans

FROM `umt-msba.transactions.transArchive_*` AS t
Join `transactions.department_lookup` AS d 
On t.department = d.department
Where   t.department NOT IN (0,
    15)
Group BY t.upc, t.description, t.department, d.dept_name, year, month
Order BY t.description, year, month

"""

In [12]:
holder3 = pandas_gbq.read_gbq(query_3,project_id=gbq_proj_id,credentials=credentials)

holder3.head()

Downloading: 100%|██████████|


,upc,description,department,dept_name,year,month,sales,Items,trans
0,0000000004220,COCKTAILS Fitz.OldFashioned,22.0,BEER & WINE,2015,2,112.00,16,17
1,0020693200000,Smoked Wild Sockeye Salmon,16.0,FISH&SEAFOOD,2014,8,272.79,55,50
2,77.67DP16,FISH&SEAFOOD,16.0,FISH&SEAFOOD,2012,5,77.67,1,1
3,0000000008028,Gift Card Added Value,15.0,MISC P/I,2013,11,1983.00,15,12
4,0020680400000,Nerka Coho Salmon Fillet,16.0,FISH&SEAFOOD,2014,6,7002.76,584,553


In [14]:
holder3.to_sql("product_sales",con,if_exists="replace",index=False)

1214088

In [15]:
con.close()